[Streams source](https://www.nvroads.com/cctv?start=0&length=10&order%5Bi%5D=1&order%5Bdir%5D=asc)

## Imports

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 14329, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 14329 (delta 78), reused 82 (delta 41), pack-reused 14194
Receiving objects: 100% (14329/14329), 13.36 MiB | 21.57 MiB/s, done.
Resolving deltas: 100% (9890/9890), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 6.7 MB/s 


## 1 Stream

In [ ]:
# !python detect.py --source https://d3wse.its.nv.gov:443/elkoxcd01/i-80_and_battle_mountain_public.stream/playlist.m3u8

In [ ]:
from detect import run

In [ ]:
run(
    source="https://d3wse.its.nv.gov:443/elkoxcd01/i-80_and_battle_mountain_public.stream/playlist.m3u8",
    save_txt=True
)

YOLOv5 🚀 2022-10-17 Python-3.7.14 torch-1.12.1+cu113 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show() image displays
cv2.imshow() is disabled in Docker environments
1/1: https://d3wse.its.nv.gov:443/elkoxcd01/i-80_and_battle_mountain_public.stream/playlist.m3u8...  Success (inf frames 352x240 at 29.97 FPS)

0: 448x640 1 car, 1 traffic light, 351.3ms
0: 448x640 2 traffic lights, 301.4ms
0: 448x640 2 traffic lights, 331.8ms
0: 448x640 2 traffic lights, 243.4ms
0: 448x640 2 traffic lights, 241.8ms
0: 448x640 2 traffic lights, 248.0ms
0: 448x640 2 traffic lights, 251.1ms
0: 448x640 2 traffic lights, 237.2ms
0: 448x640 2 traffic lights, 242.1ms
0: 448x640 2 traffic lights, 248.6ms
0: 448x640 2 traffic lights, 244.9ms
0: 448x640 2 traffic lights, 241.1ms
0: 448x640 2 traffic lights, 243.2ms
0: 448x640 2 traffic lights, 242.0ms
0: 448x640 2 traffic lights, 240.7ms
0: 448x640 2 traffic l

KeyboardInterrupt: ignored

In [ ]:
from utils.general import xywh2xyxy

In [ ]:
import pandas as pd
df=pd.read_csv('runs/detect/exp2/labels/playlist_0.txt',sep=' ',header=None)

In [ ]:
df

,0,1,2,3,4
0,2,0.884943,0.345833,0.082386,0.083333
1,9,0.967330,0.243750,0.019886,0.045833


In [ ]:
def xywh2xyxy_simplified(x, y, w, h):
    xmin = x - w / 2 # top_left_x
    ymin = y - h / 2 # top_left_y
    xmax = x + w / 2 # bottom_right_x
    ymax = y + h / 2 # bottom_right_y
    return (
        xmin,
        ymin,
        xmax,
        ymax,
    )

In [ ]:
xywh2xyxy_simplified(row[0], row[1],row[2],row[3])

(8.878125, 0.9573868, 9.121875, 0.9772732000000001)

In [ ]:
# get class names
import yaml

with open("data/coco128.yaml", "r") as f:
    config = yaml.load(f, yaml.Loader)
names = config['names']

In [ ]:
d = {
'xmin':[],
'ymin':[],
'xmax':[],
'ymax':[],
'name':[]
}
for i, row in df.iterrows():
    (
        xmin,
        ymin,
        xmax,
        ymax
    ) = xywh2xyxy_simplified(row[0], row[1],row[2],row[3])
    d['xmin'].append(xmin)
    d['ymin'].append(ymin)
    d['xmax'].append(xmax)
    d['ymax'].append(ymax)
    d['name'].append(names[int(row[0])])
    
df2 = pd.DataFrame.from_dict(d)
df3 = pd.concat([df,df2],axis=1)
df3.columns = ["class", 1, 2, 3, 4, 'xmin', 'ymin', 'xmax', 'ymax','name'] # rename class column

In [ ]:
df3

,class,1,2,3,4,xmin,ymin,xmax,ymax,name
0,2,0.884943,0.345833,0.082386,0.083333,1.827084,0.843750,2.172916,0.926136,car
1,9,0.967330,0.243750,0.019886,0.045833,8.878125,0.957387,9.121875,0.977273,traffic light


## Multiprocess Stream

In [ ]:
%cd /content/yolov5

/content/yolov5


In [ ]:
from multiprocessing import Process
import time
from detect import run

In [ ]:
streams = ["https://d3wse.its.nv.gov:443/elkoxcd01/i-80_and_battle_mountain_public.stream/playlist.m3u8",
"https://d3wse.its.nv.gov:443/elkoxcd01/i-80_and_carlin_tunnel_west_wb_thermal_public.stream/playlist.m3u8",
"https://d3wse.its.nv.gov:443/elkoxcd01/i-80_and_carlin_tunnel_east_2_public.stream/playlist.m3u8",]

In [ ]:
processes = []

def run_custom(source, name, save_txt):
    run(source=source,name=name,save_txt=save_txt)

for i, stream in enumerate(streams):
    process = Process(
        target=run_custom, name="Run stream", args=(stream, "stream" + str(i), True))
    processes.append(process)

print(f'PROCESSES: {processes}')

for process in processes:
    process.start()

time.sleep(15)

for process in processes:
    process.terminate()

PROCESSES: [<Process(Run stream, initial)>, <Process(Run stream, initial)>, <Process(Run stream, initial)>]


YOLOv5 🚀 v6.2-205-geef9057 Python-3.7.15 torch-1.12.1+cu113 CPU

YOLOv5 🚀 v6.2-205-geef9057 Python-3.7.15 torch-1.12.1+cu113 CPU

YOLOv5 🚀 v6.2-205-geef9057 Python-3.7.15 torch-1.12.1+cu113 CPU



  0%|          | 0.00/14.1M [00:00<?, ?B/s]

  0%|          | 0.00/14.1M [00:00<?, ?B/s]

  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Fusing layers... 


Fusing layers... 
Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

1/1: https://d3wse.its.nv.gov:443/elkoxcd01/i-80_and_carlin_tunnel_west_wb_thermal_public.stream/playlist.m3u8...  Success (inf frames 400x300 at 30.00 FPS)

1/1: https://d3wse.its.nv.gov:443/elkoxcd01/i-80_and_battle_mountain_public.stream/playlist.m3u8...  Success (inf frames 352x240 at 29.97 FPS)
1/1: https://d3wse.its.nv.gov:443/elkoxcd01/i-80_and_carlin_tunnel_east_2_public.stream/playlist.m3u8...  Success (inf frames 352x240 at 29.97 FPS)


0: 448x640 (no detections), 1668.4ms
0: 448x640 (no detections), 1721.8ms
0: 480x640 1 

## Get links for all streams

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.nvroads.com/cctv?start=20&length=10&order%5Bi%5D=1&order%5Bdir%5D=asc'
r = requests.get(url)

In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="/Content/NV/images/favicon/favicon.ico" rel="shortcut icon"/>
  <link href="/Content/NV/images/favicon/favicon.ico" rel="icon" sizes="16x16 32x32 64x64"/>
  <link href="/Content/NV/images/favicon/favicon-192.png" rel="icon" sizes="196x196" type="image/png"/>
  <link href="/Content/NV/images/favicon/favicon-160.png" rel="icon" sizes="160x160" type="image/png"/>
  <link href="/Content/NV/images/favicon/favicon-96.png" rel="icon" sizes="96x96" type="image/png"/>
  <link href="/Content/NV/images/favicon/favicon-64.png" rel="icon" sizes="64x64" type="image/png"/>
  <link href="/Content/NV/images/favicon/favicon-32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/Content/NV/images/favicon/favicon-16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/Content/NV/images/favicon/favicon-57.png" rel="apple-touch-icon"/>
  <link href="/Content/NV/images/favicon/favicon-114.png" rel="apple-touch-icon" sizes="114x114"